In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install transformers==4.35.0
!pip install datasets==2.14.0
!pip install spacy==3.7.2
!pip install scispacy==0.5.3
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.3/en_core_sci_md-0.5.3.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.1/123.1 kB 2.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 59.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 60.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.31.1
    Uninstalling huggingface-hub-0.31.1:
      Successfully uninstalled huggingface-hub-0.31.1
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour

In [3]:
!pip install --force-reinstall \
    transformers==4.35.0 \
    datasets==2.14.0 \
    spacy==3.7.2 \
    scispacy==0.5.3 \
    --no-deps

# ✅ Install the sciSpacy model separately
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.3/en_core_sci_md-0.5.3.tar.gz

# 🧠 Download spaCy language model again to be sure
!python -m spacy validate


  Using cached transformers-4.35.0-py3-none-any.whl.metadata (123 kB)
  Using cached datasets-2.14.0-py3-none-any.whl.metadata (19 kB)
  Using cached spacy-3.7.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (25 kB)
  Using cached scispacy-0.5.3-py3-none-any.whl.metadata (16 kB)
Using cached transformers-4.35.0-py3-none-any.whl (7.9 MB)
Using cached datasets-2.14.0-py3-none-any.whl (492 kB)
Using cached spacy-3.7.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.6 MB)
Using cached scispacy-0.5.3-py3-none-any.whl (45 kB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.0
    Uninstalling transformers-4.35.0:
      Successfully uninstalled transformers-4.35.0
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.0
    Uninstalling datasets-2.1

In [4]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.3/en_ner_bionlp13cg_md-0.5.3.tar.gz


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 9.1 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for en_ner_bionlp13cg_md: filename=en_ner_bionlp13cg_md-0.5.3-py3-none-any.whl size=119816870 sha256=e2381f1c5270cd345a74ccb6acd7753f229d81f65915e17895be257322485091
  Stored in directory: /root/.cache/pip/wheels/fd/2b/53/6d36d4c44cded3a649617560bfaaa3a9bdb3f7e3bab37e553a
Successfully built en_ner_bionlp13cg_md


In [5]:
import json

# Load structured output from Notebook 1
with open('/kaggle/input/medical-extraction/all_structured_info.json', 'r') as f:
    structured_notes = json.load(f)

print(f"✅ Loaded {len(structured_notes)} structured records.")


✅ Loaded 4966 structured records.


In [6]:
# Store all relation triples
relations_from_structured = []

for entry in structured_notes:
    sid = entry["id"]
    diagnosis_list = entry.get("Diagnosis", [])
    symptoms = entry.get("Symptoms", [])
    treatments = entry.get("Treatments", [])
    organs = entry.get("Organs", [])

    for diagnosis in diagnosis_list:
        # Symptom → indicates → Diagnosis
        for symptom in symptoms:
            relations_from_structured.append([symptom, "indicates", diagnosis])
        
        # Treatment → treats → Diagnosis
        for treatment in treatments:
            relations_from_structured.append([treatment, "treats", diagnosis])
        
        # Diagnosis → affects → Organ
        for organ in organs:
            relations_from_structured.append([diagnosis, "affects", organ])


In [7]:
import json


# ✅ Create a new structured block
structured_block = {
    "sample_id": 4966,
    "text": "Relations generated from structured_info.json (Notebook 1)",
    "relations": relations_from_structured
}

# ✅ Save as new file (no merging)
new_relations_list = [structured_block]

# 📁 Save to new file
with open("/kaggle/working/all_medical_relations_structured_only.json", "w") as f:
    json.dump(new_relations_list, f, indent=2)

# ✅ Confirmation
print(f"✅ Saved {len(relations_from_structured)} relations from structured_info to new file.")
print(f"📁 File path: /kaggle/working/all_medical_relations_structured_only.json")


✅ Saved 74900 relations from structured_info to new file.
📁 File path: /kaggle/working/all_medical_relations_structured_only.json


# Dataset 2 for disease_symptoms relation extraction

In [8]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("snmahsa/disease-symptoms-and-treatments-dataset")

print("Path to dataset files:", path)


Path to dataset files: /kaggle/input/disease-symptoms-and-treatments-dataset


In [9]:
import pandas as pd

# Load dataset
df = pd.read_csv('/kaggle/input/disease-symptoms-and-treatments-dataset/Diseases_Symptoms.csv')

# Show basic info
print("✅ Dataset shape:", df.shape)
print("\n🧪 Sample rows:")
print(df.head(3))


✅ Dataset shape: (405, 6)

🧪 Sample rows:
                       Name  \
0   Gestational Cholestasis   
1  Injury to Internal Organ   
2                   Scabies   

                                            Symptoms  \
0     Itchy skin, particularly on the hands and feet   
1        Abdominal pain, bleeding, organ dysfunction   
2  Intense itching, especially at night, small bl...   

                                          Treatments Disease_Code  Contagious  \
0                                                NaN         D001       False   
1  Immediate medical attention, diagnostic tests,...         D002       False   
2  Prescription medications (topical or oral scab...         D003       False   

   Chronic  
0    False  
1    False  
2     True  


In [10]:
import re

# Function to clean and tokenize symptom/treatment strings
def clean_and_split(text):
    if pd.isna(text):
        return []
    return [re.sub(r"[^\w\s\-]", "", item).strip().lower()
            for item in text.split(",") if item.strip()]

# Clean up fields
df["Disease"] = df["Name"].fillna("").str.strip().str.title()
df["Cleaned_Symptoms"] = df["Symptoms"].apply(clean_and_split)
df["Cleaned_Treatments"] = df["Treatments"].apply(clean_and_split)

# Drop unnecessary columns
df_cleaned = df[["Disease", "Cleaned_Symptoms", "Cleaned_Treatments"]].copy()

# Drop rows where both lists are empty
df_cleaned = df_cleaned[(df_cleaned["Cleaned_Symptoms"].str.len() > 0) | (df_cleaned["Cleaned_Treatments"].str.len() > 0)].reset_index(drop=True)

# Preview cleaned data
print("✅ Cleaned dataset shape:", df_cleaned.shape)
df_cleaned.head()


✅ Cleaned dataset shape: (405, 3)


,Disease,Cleaned_Symptoms,Cleaned_Treatments
0,Gestational Cholestasis,"[itchy skin, particularly on the hands and feet]",[]
1,Injury To Internal Organ,"[abdominal pain, bleeding, organ dysfunction]","[immediate medical attention, diagnostic tests..."
2,Scabies,"[intense itching, especially at night, small b...",[prescription medications topical or oral scab...
3,Congenital Glaucoma,"[cloudy or hazy eyes, excessive tearing, sensi...","[surgery eg, trabeculotomy, goniotomy to creat..."
4,Avoidant/Restrictive Food Intake Disorder (Arfid),[avoidance or restriction of certain foods or ...,"[nutritional counseling, psychotherapy such as..."


In [11]:
import json

# ✅ Step 1: Initialize relation list
relations_from_disease_data = []

# ✅ Step 2: Loop through each row in cleaned DataFrame and create relation triples
for idx, row in df_cleaned.iterrows():
    disease = row["Disease"]

    # Symptoms → Disease
    for symptom in row["Cleaned_Symptoms"]:
        relations_from_disease_data.append([symptom, "indicates", disease])

    # Disease → Treatments
    for treatment in row["Cleaned_Treatments"]:
        relations_from_disease_data.append([disease, "treated_by", treatment])

# ✅ Step 3: Wrap into a new block
disease_block = {
    "sample_id": 4967,
    "text": "Relations from disease–symptoms dataset (Notebook 2)",
    "relations": relations_from_disease_data
}

# ✅ Step 4: Load the previous file created from Notebook 1
with open("/kaggle/working/all_medical_relations_structured_only.json", "r") as f:
    all_relations = json.load(f)

# ✅ Step 5: Append the new relations block
all_relations.append(disease_block)

# ✅ Step 6: Save back to same file (overwrite with updated content)
with open("/kaggle/working/all_medical_relations_structured_only.json", "w") as f:
    json.dump(all_relations, f, indent=2)

# ✅ Completion Logs
print(f"✅ Appended {len(relations_from_disease_data)} relations from second dataset.")
print(f"📦 Total blocks in all_medical_relations_structured_only.json: {len(all_relations)}")


✅ Appended 3714 relations from second dataset.
📦 Total blocks in all_medical_relations_structured_only.json: 2


In [1]:
import json

with open("/kaggle/working/all_medical_relations_structured_only.json", "r") as f:
    data = json.load(f)

# Confirm relation counts per block
total = 0
for i, block in enumerate(data):
    count = len(block["relations"])
    total += count
    print(f"📦 Block {i+1} — {block['text'][:40]}... has {count} relations.")

print(f"\n✅ Grand Total: {total} relations")
print(head(total))

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/all_medical_relations_structured_only.json'